In [26]:
from __future__ import division
import hoomd
from hoomd import deprecated
from hoomd import md
import gsd.hoomd
#import settings
import random
import numpy as np
import time 

t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))

deg=30
Angle = deg*np.pi/180.
beadr=0.8
patchr=0.4
dr=beadr-patchr

def hoomdRun():
    A,B,C = com()
    sig = [1.0,1.0,1.0,0.005,1.0,1.0,1.0,1.0,0.005]
    eps = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
    types = ['1','2','3','4','5','6','7','V1','V2']

    
    
    # SETUP #
    hoomd.context.initialize("")
    rcut =  max(sig)*2.5
    
    
    ### MAKE SNAPSHOT ###
    #sn=hoomd.data.make_snapshot(N=2,box=hoomd.data.boxdim(L=40),bond_types=['tether'],particle_types=['V1','V2'])
    sn=hoomd.data.make_snapshot(N=2,box=hoomd.data.boxdim(L=40),bond_types=['tether'],particle_types=['V1','V2'])
    system=hoomd.init.read_snapshot(sn)
    system.particles.get(0).position=[3,0,0]
    system.particles.get(0).diameter=beadr
    system.particles.get(0).moment_inertia=[0.611524, 1.1476125, 1.7591365]
    
    system.particles.get(1).position=[-3,0,0]
    system.particles.get(1).diameter=beadr
    system.particles.get(1).moment_inertia=[0.611524, 1.1476125, 1.7591365]
    
    system.particles.types.add('1')
    system.particles.types.add('2')
    system.particles.types.add('3')
    system.particles.types.add('4')
    system.particles.types.add('5')
    system.particles.types.add('6')
    system.particles.types.add('7')
    
    ### ADD TETHERED PARTICLE ###
    y=system.particles.add('4')
    system.particles.get(y).position=[0,0,0]
    
    b=system.particles.add('5')
    system.particles.get(b).position=[.1,.1,.1]
    
    h=system.particles.add('7')
    system.particles.get(h).position=[.2,.2,.2]
    
    ### GIVE IT MOMENT/DIAMETER ###
    system.particles.get(y).diameter=0
    system.particles.get(y).moment_inertia=[0,0,0]

    system.particles.get(b).diameter=0
    system.particles.get(b).moment_inertia=[0,0,0]
    
    system.particles.get(h).diameter=0
    system.particles.get(h).moment_inertia=[0,0,0]
    
    ### RIGID BODIES ###
    rigid = hoomd.md.constrain.rigid()
    
    ###Sets the initial coordinates of patches on V1 to be the initial values of A, B, & C
    rigid.set_param('V1',positions=[(A[0],A[1],A[2]),(B[0],B[1],B[2]),(C[0],C[1],C[2])],
                    types=['1','2','3'],diameters=[sig[0]/2,sig[1]/2,sig[2]/2])
    
    ###Sets the initial coordinates of patches on V1 to be the initial values of A, B, & C
    rigid.set_param('V2',positions=[(A[0],A[1],A[2]),(B[0],B[1],B[2]),(C[0],C[1],C[2])],
                    types=['4','5','6'],diameters=[sig[0]/2,sig[1]/2,sig[2]/2])

    rigid.create_bodies()

    ### SET UP BONDS ###
    system.bonds.add('tether',0,y)
    system.bonds.add('tether',y,b)
    system.bonds.add('tether',b,h)
    system.bonds.add('tether',h,1)
    harm=md.bond.harmonic()
    harm.bond_coeff.set('tether',k=20,r0=0.83)
    
    
    ### GROUPS ###
    center = hoomd.group.rigid_center()
    part = hoomd.group.all()
    g_rigid = hoomd.group.rigid()

    nl = md.nlist.cell(check_period=1)
    
    
    ### SET UP LJ INTERACTIONS ###
    lj = md.pair.lj(r_cut=rcut,nlist=nl)
    for i in range(0,len(sig)):
        for j in range(i,len(sig)):
                lj.pair_coeff.set(types[i],types[j],epsilon=0.5*(eps[i]+eps[j]),sigma=0.5*(sig[i]+sig[j]))
    
    
    
    ### INTEGRATORS ###
    hoomd.md.integrate.mode_standard(dt=0.002)
    #hoomd.md.integrate.langevin(group=part, kT=0.01, seed = 5)
    hoomd.md.integrate.langevin(group=hoomd.group.nonrigid(), kT=0.01, seed = 5)
    hoomd.md.integrate.langevin(group=center, kT=0.01, seed = 5)
    
    
    
    ### DUMP AND RUN ###
    hoomd.analyze.log(filename='random.log',quantities=['temperature','num_particles','ndof',
                                                        'translational_ndof','rotational_ndof',
                                                        'potential_energy'],period=1000,header_prefix='#'
                                                          ,overwrite=True)
    hoomd.deprecated.dump.xml(group=part,filename ='random.xml',vis=True,image=True)
    hoomd.dump.gsd(filename='random %s.gsd'%t,period=1000, group = hoomd.group.all() ,overwrite = True)
    hoomd.run(1000000)

def com():
    A = [0,0,0] #coordinates A
    B = [0.7575,0.5865,0] #coordinates B
    C = [1.515,0,0] #coordinates C
    M = [1.0,16.0,1.0] #masses 
    TotalM = M[0]+M[1]+M[2] #total mass
    cX = (M[0]*A[0] + M[1]*B[0] + M[2]*C[0])/TotalM #COM coord, X
    cY = (M[0]*A[1] + M[1]*B[1] + M[2]*C[1])/TotalM # Y
    cZ = (M[0]*A[2] + M[1]*B[2] + M[2]*C[2])/TotalM # Z
    com = [cX,cY,cZ]
    cXYZ = [[A[0]-cX,A[1]-cY,A[2]-cZ],[B[0]-cX,B[1]-cY,B[2]-cZ],[C[0]-cX,C[1]-cY,C[2]-cZ]] # new coords
    Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
    for i in range(3):
        Ixx += M[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
        Iyy += M[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
        Izz += M[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
        Ixy += -M[i]*cXYZ[i][0]*cXYZ[i][1]
        Ixz += -M[i]*cXYZ[i][0]*cXYZ[i][2]
        Iyz += -M[i]*cXYZ[i][1]*cXYZ[i][2]
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        print(Idiag)
        comX = 0-com[0]
        comY = 0-com[1]
    A = [A[0]+comX,A[1]+comY,A[2]]
    B = [B[0]+comX,B[1]+comY,B[2]]
    C = [C[0]+comX,C[1]+comY,C[2]]
    return A,B,C

hoomdRun()

(array([0.        , 0.84559469, 0.84559469]), matrix([[-0.82376154,  0.56693644,  0.        ],
        [-0.56693644, -0.82376154,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]))
(array([0.04488356, 0.86865824, 0.91354181]), matrix([[-0.80129398,  0.59827081,  0.        ],
        [-0.59827081, -0.80129398,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]))
(array([0.611524 , 1.1476125, 1.7591365]), matrix([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))
notice(2): Group "all" created containing 2 particles
notice(2): constrain.rigid(): Creating 2 rigid bodies (adding 6 particles)
notice(2): Group "rigid_center" created containing 2 particles
notice(2): Group "rigid" created containing 8 particles
notice(2): Group "nonrigid" created containing 3 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2)

*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group


In [20]:
σ=5
print(σ)

5


In [2]:
system.get_typeid_mapping()

NameError: name 'system' is not defined

In [10]:
print(system.particles)

NameError: name 'system' is not defined